In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 30 01:02:50 2022

@author: parkh
"""

import random
import copy
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
import plotly.express as px
from datetime import datetime
import matplotlib.pyplot as plt
params = {
    'MUT': 1,  # 변이확률(%)
    'END' : 0.9,  # 설정한 비율만큼 chromosome이 수렴하면 탐색을 멈추게 하는 파라미터 (%)
    'POP_SIZE' : 1000,  # population size 10 ~ 100
    'RANGE' : 10, # chromosome의 표현 범위, 만약 10이라면 00000 00000 ~ 11111 11111까지임
    'NUM_OFFSPRING' : 5, # 한 세대에 발생하는 자식 chromosome의 수
    'SELECTION_PRESSURE' : 3, # 선택연산의 선택압
    'job_seq' : [y for x in range(1,5) for y in range(1,13)],
    'factory_seq' : [1,2]
    # 원하는 파라미터는 여기에 삽입할 것
    }
class JAYA_FJSP():
    def __init__(self, parameters):
        self.eps = 0.5
        self.eps2 = 0.1
        self.eps3 = 0.2
        self.params = {}
        for key, value in parameters.items():
            self.params[key] = value
        self.p_table = pd.read_csv('C:/Users/parkh/git_tlsgudcks/FJSP/data/FJSP_SIM4.csv',index_col=(0)) #job과 operation을 기록한 테이블
        self.s_table = pd.read_csv('C:/Users/parkh/git_tlsgudcks/FJSP/data/FJSP_SETUP_SIM.csv', index_col=(0)) #setup time 테이블
        self.job_endTime={'j1':0, 'j2':0, 'j3':0, 'j4':0, 'j5':0, 'j6':0,'j7':0, 'j8':0, 'j9':0, 'j10':0, 'j11':0, 'j12':0} # job의 끝나는 지점을 등록
        self.machine_endTime={'M1':0,'M2':0,'M3':0,'M4':0} # machine의 끝나는 지점을 등록
        self.machine_prejob={'M1':"j0", 'M2':"j0",'M3':"j0", 'M4':"j0",'M5':"j0", 'M6':"j0",'M7':"j0", 'M8':"j0"}
        self.job_preOperation={'1':1,'2':1,'3':1,'4':1,'5':1,'6':1,'7':1,'8':1,'9':1,'10':1, '11':1, '12':1}
        self.job_max_op = {'1':4, '2':4, '3':4, '4':4, '5':4, '6':4, '7':4, '8':4, '9':4, '10':4, '11':4, '12':4}
        
    def reset(self):
        self.job_endTime={'j1':0, 'j2':0, 'j3':0, 'j4':0, 'j5':0, 'j6':0,'j7':0, 'j8':0, 'j9':0, 'j10':0, 'j11':0, 'j12':0} # job의 끝나는 지점을 등록
        self.machine_endTime={'M1':0,'M2':0,'M3':0,'M4':0} # machine의 끝나는 지점을 등록
        self.machine_prejob={'M1':"j0", 'M2':"j0",'M3':"j0", 'M4':"j0",'M5':"j0", 'M6':"j0",'M7':"j0", 'M8':"j0"}
        self.job_preOperation={'1':1,'2':1,'3':1,'4':1,'5':1,'6':1,'7':1,'8':1,'9':1,'10':1, '11':1, '12':1}
    
    def operation_check(self, job_type):
        if job_type < 10:
            job = "j0"+str(job_type)
        else:
            job = "j" +str(job_type)
        if self.job_preOperation[str(job_type)] < 10:
            operation = "0"+str(self.job_preOperation[str(job_type)])
        else:
            operation = str(self.job_preOperation[str(job_type)])
        jop = job+operation
        
        return jop
    def gannt_chart(self, population):
        assignment = []
        for j in range(48):
            assignment.append([population[0][0][j],population[0][1][j]])
        print(assignment)
        
        plotlydf = pd.DataFrame([],columns=['Task','Start','Finish','Resource']) #간트차트로 보여주기 위한 데이터프레임
        i=0 #간트차트의 인덱싱을 위한 숫자
        j=0
        for job_num,machine in assignment:  #['j11','M2']의 형태에서 잡과 머신을 가져옴
            job = 'j'+str(job_num)        #'j11'의 형태를 j1로 
            job_op = self.operation_check(job_num)
            self.job_preOperation[str(job_num)] += 1
            df2_sorted = self.s_table[job] #셋업테이블에서 job에 해당하는 컬럼을 가져옴
            setup_time=df2_sorted.loc[self.machine_prejob[machine]] #컬럼에서 machine에 세팅되어있던 job에서 변경유무 확인
            time = max(self.machine_endTime[machine] ,self.job_endTime[job]) #machine과 job의 순서 제약조건을 지키기 위해 더 큰 값을 설정함
            df_sorted = self.p_table[machine] #p_time테이블에서 현재 machine에 해당하는 열을 가져옴
            p_time = df_sorted.loc[job_op] #해당하는 job과 operation의 시간을 가져옴
            start = datetime.fromtimestamp(time*3600) #포매팅 해줌
            time = time+setup_time # 프로세스타임과 셋업타임을 더해줌
            p_start=datetime.fromtimestamp(time*3600)
            time = time+p_time
            end = datetime.fromtimestamp(time*3600) #끝나는 시간 포매팅
            plotlydf.loc[j] = dict(Task=job, Start=p_start, Finish=end, Resource=machine) #간트차트를 위한 딕셔너리 생성, 데이터프레임에 집어넣음
            if setup_time !=0:
                j+=1
                plotlydf.loc[j] = dict(Task="setup", Start=start, Finish=p_start, Resource=machine) #간트차트를 위한 딕셔너리 생성, 데이터프레임에 집어넣음
            j += 1
            i += 1 #데이터 프레임 인덱싱 증가
            self.machine_endTime[machine]=time #기계의 끝나는 시간 설정
            self.job_endTime[job]=time #job의 끝나는 시간 설정
            self.machine_prejob[machine] = job #현재 어떤 machine에서 어떤 job을 수행했는지 기록      
        self.reset()
        plotlydf2 = plotlydf.sort_values(by=['Resource','Task'], ascending=False)
        fig = px.timeline(plotlydf2, x_start="Start", x_end="Finish", y="Resource", color="Task", width=1000, height=400)
        fig.show()
    def sco(self, dad, mom):
        dad_ch = copy.deepcopy(dad)
        mom_ch = copy.deepcopy(mom)
        point = random.randint(0,47)
        os_offspring = [-1 for i in range(48)]
        for i in range(point):
            os_offspring[i] = dad_ch[0][i]
            mom_ch[0].remove(os_offspring[i])
        for i in range(48):
            if os_offspring[i] == -1:
                os_offspring[i] = mom_ch[0].pop(0)
        ma_offspring = self.init_MA_LS(os_offspring)
        c_max, critical_machine, flow_time, util = self.get_fittness(os_offspring, ma_offspring)
        offspring = [os_offspring, ma_offspring, c_max, critical_machine, flow_time, util ]
        return offspring
    def jco(self, dad, mom):
        dad_ch = copy.deepcopy(dad)
        mom_ch = copy.deepcopy(mom)
        job_list = [x for x in range(1,13)]
        for i in range(1,13):
            coin = random.random()
            if coin >0.5:
                job_list.remove(i)
        os_offspring = [-1 for i in range(48)]
        for i in range(48):
            if dad_ch[0][i] in job_list:
                os_offspring[i] = dad_ch[0][i]
                mom_ch[0].remove(dad_ch[0][i])
        for i in range(48):
            if os_offspring[i] == -1:
                os_offspring[i] = mom_ch[0].pop(0)
        ma_offspring = self.init_MA_LS(os_offspring)
        c_max, critical_machine, flow_time, util = self.get_fittness(os_offspring, ma_offspring)
        offspring = [os_offspring, ma_offspring, c_max, critical_machine, flow_time, util ]
        return offspring
    def aco(self, dad, mom):
        dad_ch = copy.deepcopy(dad)
        mom_ch = copy.deepcopy(mom)
        os_offspring = copy.deepcopy(dad_ch[0])
        ma_offspring = [-1 for i in range(48)]
        for i in range(48):
            for j in range(48):
               if os_offspring[i] == mom_ch[0][j]:
                   mom_ch[0][j] = 0
                   ma_offspring[i] = mom_ch[1][j]
                   break
        c_max, critical_machine, flow_time, util = self.get_fittness(os_offspring, ma_offspring)
        offspring = [os_offspring, ma_offspring, c_max, critical_machine, flow_time, util ]
        return offspring
    def get_fittness(self,scheduling_seq,routing_seq):
        time_list=[]
        p_time_list = []
        for j in range(48):
            job_number = scheduling_seq[j]
            machine = routing_seq[j]
            job = "j"+str(job_number)
            jobOp = self.operation_check(job_number)
            self.job_preOperation[str(job_number)] += 1
            
            setup_list = self.s_table[job]
            setup_time=setup_list.loc[self.machine_prejob[machine]]
            start_time = max(self.machine_endTime[machine] ,self.job_endTime[job])
            p_list = self.p_table[machine]
            p_time = p_list.loc[jobOp]
            p_time_list.append(p_time)
            end_time = start_time +setup_time+p_time
            time_list.append([start_time,end_time])
            self.machine_endTime[machine]=end_time #기계의 끝나는 시간 설정
            self.job_endTime[job]=end_time #job의 끝나는 시간 설정
            self.machine_prejob[machine] = job #현재 어떤 machine에서 어떤 job을 수행했는지 기록
        all_values = self.machine_endTime.values()
        all_values2 = self.job_endTime.values()
        all_time = sum(all_values)
        flow_time = sum(all_values2)
        value_add_time = sum(p_time_list)
        util = value_add_time / all_time
        c_max=max(all_values)
        all_values = list(all_values)
        k=0
        critical_machine=""
        for i in range(4):
            if all_values[i] > k:
                k=all_values[i]
                critical_machine = str(i+1)
        return c_max, critical_machine, flow_time, util
    def anneal_eps(self):
        self.eps -=0.001
        self.eps2 +=0.001
        self.eps2 = min(self.eps2, 0.6)
        self.eps3 +=0.002
        self.eps3 = min(self.eps3, 0.8) 
    def init_OS(self):
        random.shuffle(self.params['job_seq'])
        scheduling_seq=copy.deepcopy(self.params['job_seq'])
        return scheduling_seq
    def init_MA_Random(self, scheduling_seq): # 이게 걍 랜덤하게 초기화
        routing_seq=[]
        for operation in scheduling_seq:
            operation2 = self.operation_check(operation)
            self.job_preOperation[str(operation)] += 1
            for i in self.machine_endTime:
                a = self.p_table[i].loc[operation2]
                if a != 0:
                    routing_seq.append(i)
                    break
        return routing_seq
    def init_MA_LS(self, scheduling_seq): #이건 SPT초기화
        routing_seq=[]
        for operation in scheduling_seq:
            operation2 = self.operation_check(operation)
            self.job_preOperation[str(operation)] += 1
            machine = self.least_time_machine(operation,operation2)
            routing_seq.append(machine)
        self.reset()
        return routing_seq
    def AAM(self, offspring): #local하게 좋은 
        coin = random.randint(1, 1)
        scheduling_seq ,routing_seq, fittness, critical_machine, flow_time, util = offspring
        if coin == 1:
            routing_seq2 = copy.deepcopy(routing_seq)
            critical_machine_index = []
            machine = "M"+str(critical_machine)#과부하 걸린 기계
            for i in range(48): #과부하 걸린 기계의 순서를 저장해놓음
                if machine == routing_seq[i]:
                    critical_machine_index.append(i) #[1,13,20,21,24,30,38]
            random.shuffle(critical_machine_index)   
            job = scheduling_seq[critical_machine_index[0]] # 24의 7이었음
            k=0
            for i in range(critical_machine_index[0]+1): #24까지 돌면서 7이 몇번나오나
                if job == scheduling_seq[i]:
                    k+=1
            if job>9:
                job_op = "j"+str(job)+"0"+str(k) #72
            else:
                job_op = "j0"+str(job)+"0"+str(k) #72
            machine = self.random_machine(job, job_op)
            routing_seq2[critical_machine_index[0]] = machine
            self.reset()
            fittness, critical_machine, flow_time, util = self.get_fittness(scheduling_seq, routing_seq2)
            solution = [scheduling_seq, routing_seq2, fittness, critical_machine,flow_time,util]
        return solution
    def least_time_machine(self, job,operation2): #그 시점에서 가장 작은 machine 선택해주는거
        best_machine = ""
        max_endTime=10000
        for i in self.machine_endTime:
            time = max(self.machine_endTime[i] ,self.job_endTime["j"+str(job)])
            df_sorted = self.p_table[i]
            p_time = df_sorted.loc[operation2]
            if p_time != 0:
                df2_sorted = self.s_table["j"+str(job)]
                setup_time=df2_sorted.loc[self.machine_prejob[i]]
                start = time
                end = start+setup_time+p_time
                if max_endTime>end:
                    max_endTime = end
                    best_machine = i
        self.machine_endTime[best_machine]=max_endTime #기계의 끝나는 시간 설정
        self.job_endTime[job]=max_endTime #job의 끝나는 시간 설정
        self.machine_prejob[best_machine] = "j"+str(job) #현재 어떤 machine에서 어떤 job을 수행했는지 기록
        return best_machine
    def random_machine(self, job,operation2): #그 시점에서 가장 작은 machine 선택해주는거
        random_machine = ""
        for i in self.machine_endTime:
            df_sorted = self.p_table[i]
            p_time = df_sorted.loc[operation2]
            if p_time != 0:
                random_machine = i
                break
        return random_machine
    def Two_Point_OS_LS(self, solution): #LS 세가지
        coin = random.randint(0,2)
        scheduling_seq, routing_seq, fittness, critical_machine, flow_time, util = solution
        s_v = copy.deepcopy(scheduling_seq)
        r_v = copy.deepcopy(routing_seq)
        operation_seq = [i for i in range(48)]
        k = random.choice(operation_seq)
        a = operation_seq.pop(k)
        b = random.choice(operation_seq)
        if a > b:
            a,b = b,a
        s_v2 = scheduling_seq[a:b+1]
        r_v2 = routing_seq[a:b+1]
        s_v3 = copy.deepcopy(s_v2)
        if coin == 0:
            s_v3.reverse()
            r_v3 = []
            for i in range(b-a+1):
                for j in range(b-a+1):
                    if s_v3[i] == s_v2[j]:
                        r_v3.append(r_v2[j])
                        s_v2[j] = 0
                        break
            k=0
            for i in range(a,b+1):
                s_v[i] = s_v3[k]
                r_v[i] = r_v3[k]
                k+=1
            c_max, critical_machine2,flow_time, util = self.get_fittness(s_v, r_v)
            solution = [s_v ,r_v, c_max, critical_machine2, flow_time, util]
            return solution
        elif coin == 1:
            s_v3[0] , s_v3[-1] = s_v3[-1], s_v3[0]
            r_v3 = []
            for i in range(b-a+1):
                for j in range(b-a+1):
                    if s_v3[i] == s_v2[j]:
                        r_v3.append(r_v2[j])
                        s_v2[j] = 0
                        break
            k=0
            for i in range(a,b+1):
                s_v[i] = s_v3[k]
                r_v[i] = r_v3[k]
                k+=1
            c_max, critical_machine2,flow_time, util = self.get_fittness(s_v, r_v)
            solution = [s_v ,r_v, c_max, critical_machine2, flow_time, util]
            return solution
        elif coin == 2:
            k = s_v3.pop(-1)
            s_v3.insert(0,k)
            r_v3 = []
            for i in range(b-a+1):
                for j in range(b-a+1):
                    if s_v3[i] == s_v2[j]:
                        r_v3.append(r_v2[j])
                        s_v2[j] = 0
                        break
            k=0
            for i in range(a,b+1):
                s_v[i] = s_v3[k]
                r_v[i] = r_v3[k]
                k+=1
            c_max, critical_machine2,flow_time, util = self.get_fittness(s_v, r_v)
            solution = [s_v ,r_v, c_max, critical_machine2, flow_time, util]
            return solution
    
    def replacement_operator2(self, population, offsprings):
        result_population = population[:]
        for i in range(5):
            fitness = offsprings[i][2]
            for j in range(100):
                if population[j][2] > fitness:
                    result_population[j] = copy.deepcopy(offsprings[i])
                    break
        return result_population
    def sort_population(self, population):
        population.sort(key=lambda x:x[2],reverse=False)
        # todo: fitness를 기준으로 population을 내림차순 정렬하고 반환
        return population
    def selection_operater(self, population):
        mom_ch = []
        dad_ch = []
        fitness_population=copy.deepcopy(population) #리스트 복사
        total_score = 0 #유전자의 적합도 총합 계산
        sum_fitness = 0 #0부터 적합도들의 합을 더함 이것이 k보다 클 시 그 해를 선택
        #유전자별 적합도를 새로운 리스트에 입력
        for i in range(len(population)):
            fitness_population[i][2]=abs((population[-1][2]-population[i][2])-(population[0][2]-population[-1][2])/(3-1))
        #(Cw-Ci)+(Cb-Cw)/(k-1)
        
        #유전자의 적합도 총합계산
        for i in range(len(fitness_population)):
            total_score=fitness_population[i][2]+total_score
        total_score=int(total_score)
        #랜덤함수 호출        
        k=random.uniform(0, total_score)
        #룰렛 돌리기
        for i in range(len(fitness_population)):
            sum_fitness=sum_fitness+fitness_population[i][2]
            if sum_fitness>=k:
                dad_ch=population[i]
                fitness_population.pop(i)
                break
        #적합도 총합을 다시계산해줌, 더했던 적합도들의 합도 다시 계산
        sum_fitness=0
        total_score=0
        for i in range(len(fitness_population)):
            total_score=fitness_population[i][2]+total_score
        total_score=int(total_score)
        #랜덤함수 실행
        k2=random.uniform(0, total_score)
        #룰렛 돌리기
        for i in range(len(fitness_population)):
            sum_fitness=sum_fitness+fitness_population[i][2]
            if sum_fitness>=k2:
                mom_ch=population[i]
                break
        return mom_ch, dad_ch
    def replacement_operator(self, population, offsprings):
        # todo: 생성된 자식해들(offsprings)을 이용하여 기존 해집단(population)의 해를 대치하여 새로운 해집단을 return
        """
        세대형 유전 알고리즘 사용
        해집단 내에서 가장 품질이 낮은 해를 대치하는 방법 사용(엘리티즘)
        """
        result_population = []
        for i in range(5):
            population.pop()
        for i in range(5):
            population.append(offsprings[i])
        result_population=population[:]
        return result_population
    def tournament_selection(self, population):
        k=8
        tournament_list = []
        tournament_list_4 = []
        tournament_list_2 = []
        for i in range(k):
            tournament_list.append(random.choice(population))
        for i in range(0,7,2):
            if tournament_list[i][2] < tournament_list[i+1][2]:
                tournament_list_4.append(tournament_list[i])
            else:
                tournament_list_4.append(tournament_list[i+1])
        for i in range(0,3,2):
            if tournament_list_4[i][2] < tournament_list_4[i+1][2]:
                tournament_list_2.append(tournament_list_4[i])
            else:
                tournament_list_2.append(tournament_list_4[i+1])
        mom_ch = copy.deepcopy(tournament_list_2[0])
        dad_ch = copy.deepcopy(tournament_list_2[1])
        return mom_ch, dad_ch
    def print_average_fitness(self, population):
        # todo: population의 평균 fitness를 출력
        population_average_fitness = 0
        total_population=0 
        for i in range(100):
            total_population=total_population+population[i][2]
        population_average_fitness=total_population/100
        print("population 평균 fitness: {}".format(population_average_fitness))
        return population_average_fitness
    def search(self):
        generation = 0  # 현재 세대 수
        population = [] # 해집단
        offsprings = [] # 자식해집단
        all_list = []                    
        for i in range(100):
            scheduling_seq = self.init_OS()
            routing_seq_m = self.init_MA_LS(scheduling_seq)
            self.reset()
            fittness,critical_machine,flow_time,util = self.get_fittness(scheduling_seq,routing_seq_m)
            self.reset()
            population.append([scheduling_seq,routing_seq_m,fittness,critical_machine, flow_time, util])
        population = self.sort_population(population)
        result=[]
        while True:
            offsprings = []
            count_end=0 #동일 갯수
            for i in range(5):
                mom_ch, dad_ch = self.selection_operater(population)
                self.reset()
                coin = random.randint(0,1)
                if coin == 0 :
                    offspring = self.sco(dad_ch, mom_ch)
                    self.reset()
                elif coin == 1:
                    offspring = self.jco(dad_ch, mom_ch)
                    self.reset()
                coin2 = random.random()
                if coin2 < 0.15:
                    offspring = self.Two_Point_OS_LS(offspring)
                offsprings.append(offspring)
            population = self.replacement_operator(population, offsprings)
            self.reset()
            population = self.sort_population(population)
            print('현재 세대',generation, '최고 해', population[0][2])
            avg = self.print_average_fitness(population)
            result.append([population[0][2],avg])
            self.anneal_eps()
            generation=generation+1
            pop_list=[]
            for i in range(100):
                pop_list.append(population[i][2])
            a = Counter(pop_list)
            end_number_list=[]
            for k in a.values():
                end_number_list.append(k)
            end_number = max(end_number_list)
            if end_number >= 90:
                self.reset()
                print(generation)
                break
            if generation == 1000:
                print(generation)
                break
        self.gannt_chart(population)
        return population, result
total_makespan=0
solution_list = []
makespan_list = []
util_list = []
flow_time_list = []
starttime = datetime.now()
for i in range(20):
    if __name__ == "__main__":
        jaya = JAYA_FJSP(params)
        population,result = jaya.search()
        print(population[0])
        total_makespan += population[0][2]
        makespan_list.append(population[0][2])
        util_list.append(population[0][5])
        flow_time_list.append(population[0][4])
        solution_list.append([population[0][2],population[0][4],population[0][5]])
endtime = datetime.now()
elapsed_time = endtime-starttime
total_elapsed_time = 0
total_elapsed_time += elapsed_time.total_seconds()
print(solution_list)
print("100회 실행 평균 최적 makespan: ", total_makespan/5)
print("100회 평균 총 걸린시간: ", total_elapsed_time/5)
print("5회 최소값", min(makespan_list))
print("5회 최댓값", max(makespan_list))



현재 세대 0 최고 해 163
population 평균 fitness: 197.9
현재 세대 1 최고 해 163
population 평균 fitness: 197.07
현재 세대 2 최고 해 163
population 평균 fitness: 195.57
현재 세대 3 최고 해 163
population 평균 fitness: 194.25
현재 세대 4 최고 해 163
population 평균 fitness: 193.48
현재 세대 5 최고 해 163
population 평균 fitness: 192.47
현재 세대 6 최고 해 163
population 평균 fitness: 191.28
현재 세대 7 최고 해 163
population 평균 fitness: 190.41
현재 세대 8 최고 해 163
population 평균 fitness: 190.31
현재 세대 9 최고 해 163
population 평균 fitness: 189.29
현재 세대 10 최고 해 163
population 평균 fitness: 188.3
현재 세대 11 최고 해 163
population 평균 fitness: 187.81
현재 세대 12 최고 해 163
population 평균 fitness: 187.45
현재 세대 13 최고 해 163
population 평균 fitness: 187.33
현재 세대 14 최고 해 163
population 평균 fitness: 187.07
현재 세대 15 최고 해 163
population 평균 fitness: 186.42
현재 세대 16 최고 해 163
population 평균 fitness: 186.17
현재 세대 17 최고 해 163
population 평균 fitness: 185.79
현재 세대 18 최고 해 163
population 평균 fitness: 185.88
현재 세대 19 최고 해 163
population 평균 fitness: 185.06
현재 세대 20 최고 해 163
population 평균 fitness: 184.77
현재 세

현재 세대 170 최고 해 161
population 평균 fitness: 169.06
현재 세대 171 최고 해 161
population 평균 fitness: 168.75
현재 세대 172 최고 해 161
population 평균 fitness: 168.83
현재 세대 173 최고 해 161
population 평균 fitness: 168.9
현재 세대 174 최고 해 161
population 평균 fitness: 168.91
현재 세대 175 최고 해 161
population 평균 fitness: 168.6
현재 세대 176 최고 해 161
population 평균 fitness: 169.11
현재 세대 177 최고 해 161
population 평균 fitness: 168.62
현재 세대 178 최고 해 161
population 평균 fitness: 168.86
현재 세대 179 최고 해 161
population 평균 fitness: 168.45
현재 세대 180 최고 해 161
population 평균 fitness: 169.19
현재 세대 181 최고 해 161
population 평균 fitness: 168.58
현재 세대 182 최고 해 161
population 평균 fitness: 169.27
현재 세대 183 최고 해 161
population 평균 fitness: 168.87
현재 세대 184 최고 해 161
population 평균 fitness: 168.65
현재 세대 185 최고 해 161
population 평균 fitness: 168.98
현재 세대 186 최고 해 161
population 평균 fitness: 168.54
현재 세대 187 최고 해 161
population 평균 fitness: 168.85
현재 세대 188 최고 해 161
population 평균 fitness: 168.17
현재 세대 189 최고 해 161
population 평균 fitness: 168.39
현재 세대 190 최고 해 161
pop

현재 세대 339 최고 해 157
population 평균 fitness: 164.44
현재 세대 340 최고 해 157
population 평균 fitness: 164.01
현재 세대 341 최고 해 157
population 평균 fitness: 163.65
현재 세대 342 최고 해 157
population 평균 fitness: 164.44
현재 세대 343 최고 해 157
population 평균 fitness: 163.77
현재 세대 344 최고 해 157
population 평균 fitness: 164.36
현재 세대 345 최고 해 157
population 평균 fitness: 164.0
현재 세대 346 최고 해 157
population 평균 fitness: 163.83
현재 세대 347 최고 해 157
population 평균 fitness: 164.39
현재 세대 348 최고 해 157
population 평균 fitness: 163.99
현재 세대 349 최고 해 157
population 평균 fitness: 163.4
현재 세대 350 최고 해 157
population 평균 fitness: 163.51
현재 세대 351 최고 해 157
population 평균 fitness: 163.76
현재 세대 352 최고 해 154
population 평균 fitness: 164.13
현재 세대 353 최고 해 154
population 평균 fitness: 163.4
현재 세대 354 최고 해 154
population 평균 fitness: 164.17
현재 세대 355 최고 해 154
population 평균 fitness: 163.02
현재 세대 356 최고 해 154
population 평균 fitness: 163.74
현재 세대 357 최고 해 154
population 평균 fitness: 163.39
현재 세대 358 최고 해 154
population 평균 fitness: 163.67
현재 세대 359 최고 해 154
popu

현재 세대 507 최고 해 153
population 평균 fitness: 155.28
현재 세대 508 최고 해 153
population 평균 fitness: 155.72
현재 세대 509 최고 해 153
population 평균 fitness: 155.28
현재 세대 510 최고 해 153
population 평균 fitness: 155.85
현재 세대 511 최고 해 153
population 평균 fitness: 154.88
현재 세대 512 최고 해 153
population 평균 fitness: 154.89
현재 세대 513 최고 해 153
population 평균 fitness: 155.04
현재 세대 514 최고 해 153
population 평균 fitness: 154.53
현재 세대 515 최고 해 153
population 평균 fitness: 154.45
현재 세대 516 최고 해 153
population 평균 fitness: 154.77
현재 세대 517 최고 해 153
population 평균 fitness: 154.15
현재 세대 518 최고 해 153
population 평균 fitness: 153.93
현재 세대 519 최고 해 153
population 평균 fitness: 154.25
현재 세대 520 최고 해 153
population 평균 fitness: 154.25
현재 세대 521 최고 해 153
population 평균 fitness: 153.62
현재 세대 522 최고 해 153
population 평균 fitness: 154.09
현재 세대 523 최고 해 153
population 평균 fitness: 153.6
현재 세대 524 최고 해 153
population 평균 fitness: 153.87
현재 세대 525 최고 해 153
population 평균 fitness: 153.58
현재 세대 526 최고 해 153
population 평균 fitness: 153.54
현재 세대 527 최고 해 153
po

[[5, 7, 7, 2, 3, 4, 5, 4, 10, 11, 5, 9, 12, 8, 8, 12, 5, 6, 1, 3, 2, 6, 8, 11, 2, 10, 4, 12, 11, 10, 3, 2, 9, 1, 7, 10, 9, 4, 12, 8, 6, 11, 1, 6, 7, 1, 9, 3], ['M3', 'M2', 'M1', 'M1', 'M4', 'M2', 'M3', 'M2', 'M1', 'M4', 'M3', 'M4', 'M2', 'M2', 'M2', 'M1', 'M3', 'M2', 'M3', 'M1', 'M4', 'M2', 'M3', 'M1', 'M4', 'M2', 'M2', 'M1', 'M1', 'M4', 'M3', 'M2', 'M1', 'M3', 'M3', 'M4', 'M1', 'M2', 'M1', 'M3', 'M1', 'M2', 'M4', 'M1', 'M4', 'M2', 'M3', 'M4'], 153, '1', 1557, 0.8465346534653465]
현재 세대 0 최고 해 173
population 평균 fitness: 197.78
현재 세대 1 최고 해 173
population 평균 fitness: 196.91
현재 세대 2 최고 해 173
population 평균 fitness: 196.01
현재 세대 3 최고 해 173
population 평균 fitness: 195.07
현재 세대 4 최고 해 173
population 평균 fitness: 193.66
현재 세대 5 최고 해 173
population 평균 fitness: 192.5
현재 세대 6 최고 해 173
population 평균 fitness: 191.88
현재 세대 7 최고 해 173
population 평균 fitness: 191.12
현재 세대 8 최고 해 173
population 평균 fitness: 190.59
현재 세대 9 최고 해 173
population 평균 fitness: 190.53
현재 세대 10 최고 해 167
population 평균 fitness: 189.2

...

현재 세대 160 최고 해 161
population 평균 fitness: 166.91
현재 세대 161 최고 해 161
population 평균 fitness: 166.98
현재 세대 162 최고 해 161
population 평균 fitness: 167.41
현재 세대 163 최고 해 161
population 평균 fitness: 167.38
현재 세대 164 최고 해 161
population 평균 fitness: 167.41
현재 세대 165 최고 해 161
population 평균 fitness: 166.9
현재 세대 166 최고 해 161
population 평균 fitness: 166.77
현재 세대 167 최고 해 161
population 평균 fitness: 166.76
현재 세대 168 최고 해 161
population 평균 fitness: 166.59
현재 세대 169 최고 해 161
population 평균 fitness: 166.79
현재 세대 170 최고 해 161
population 평균 fitness: 166.78
현재 세대 171 최고 해 161
population 평균 fitness: 166.92
현재 세대 172 최고 해 161
population 평균 fitness: 166.66
현재 세대 173 최고 해 161
population 평균 fitness: 166.87
현재 세대 174 최고 해 161
population 평균 fitness: 167.14
현재 세대 175 최고 해 161
population 평균 fitness: 166.7
현재 세대 176 최고 해 161
population 평균 fitness: 166.3
현재 세대 177 최고 해 161
population 평균 fitness: 166.29
현재 세대 178 최고 해 161
population 평균 fitness: 166.56
현재 세대 179 최고 해 161
population 평균 fitness: 166.27
현재 세대 180 최고 해 161
popu

현재 세대 329 최고 해 158
population 평균 fitness: 162.61
현재 세대 330 최고 해 158
population 평균 fitness: 162.65
현재 세대 331 최고 해 158
population 평균 fitness: 162.74
현재 세대 332 최고 해 158
population 평균 fitness: 163.09
현재 세대 333 최고 해 158
population 평균 fitness: 163.34
현재 세대 334 최고 해 158
population 평균 fitness: 162.54
현재 세대 335 최고 해 158
population 평균 fitness: 162.36
현재 세대 336 최고 해 158
population 평균 fitness: 162.72
현재 세대 337 최고 해 158
population 평균 fitness: 162.62
현재 세대 338 최고 해 158
population 평균 fitness: 163.05
현재 세대 339 최고 해 158
population 평균 fitness: 162.57
현재 세대 340 최고 해 158
population 평균 fitness: 162.29
현재 세대 341 최고 해 158
population 평균 fitness: 162.38
현재 세대 342 최고 해 158
population 평균 fitness: 162.35
현재 세대 343 최고 해 158
population 평균 fitness: 162.36
현재 세대 344 최고 해 158
population 평균 fitness: 162.38
현재 세대 345 최고 해 158
population 평균 fitness: 163.11
현재 세대 346 최고 해 158
population 평균 fitness: 162.37
현재 세대 347 최고 해 158
population 평균 fitness: 162.4
현재 세대 348 최고 해 158
population 평균 fitness: 162.45
현재 세대 349 최고 해 158
po

현재 세대 498 최고 해 155
population 평균 fitness: 159.93
현재 세대 499 최고 해 155
population 평균 fitness: 159.85
현재 세대 500 최고 해 155
population 평균 fitness: 159.83
현재 세대 501 최고 해 155
population 평균 fitness: 159.92
현재 세대 502 최고 해 155
population 평균 fitness: 159.88
현재 세대 503 최고 해 155
population 평균 fitness: 159.73
현재 세대 504 최고 해 155
population 평균 fitness: 159.66
현재 세대 505 최고 해 155
population 평균 fitness: 159.79
현재 세대 506 최고 해 155
population 평균 fitness: 160.01
현재 세대 507 최고 해 155
population 평균 fitness: 159.35
현재 세대 508 최고 해 155
population 평균 fitness: 160.04
현재 세대 509 최고 해 155
population 평균 fitness: 159.61
현재 세대 510 최고 해 155
population 평균 fitness: 159.5
현재 세대 511 최고 해 155
population 평균 fitness: 159.79
현재 세대 512 최고 해 155
population 평균 fitness: 160.96
현재 세대 513 최고 해 155
population 평균 fitness: 159.81
현재 세대 514 최고 해 155
population 평균 fitness: 159.64
현재 세대 515 최고 해 155
population 평균 fitness: 159.59
현재 세대 516 최고 해 155
population 평균 fitness: 159.77
현재 세대 517 최고 해 155
population 평균 fitness: 159.42
현재 세대 518 최고 해 155
po

현재 세대 667 최고 해 153
population 평균 fitness: 157.22
현재 세대 668 최고 해 153
population 평균 fitness: 157.17
현재 세대 669 최고 해 153
population 평균 fitness: 156.93
현재 세대 670 최고 해 153
population 평균 fitness: 156.88
현재 세대 671 최고 해 153
population 평균 fitness: 157.89
현재 세대 672 최고 해 153
population 평균 fitness: 157.21
현재 세대 673 최고 해 153
population 평균 fitness: 157.32
현재 세대 674 최고 해 153
population 평균 fitness: 157.13
현재 세대 675 최고 해 153
population 평균 fitness: 157.28
현재 세대 676 최고 해 153
population 평균 fitness: 157.33
현재 세대 677 최고 해 153
population 평균 fitness: 157.16
현재 세대 678 최고 해 153
population 평균 fitness: 157.37
현재 세대 679 최고 해 153
population 평균 fitness: 157.35
현재 세대 680 최고 해 153
population 평균 fitness: 156.73
현재 세대 681 최고 해 153
population 평균 fitness: 157.47
현재 세대 682 최고 해 153
population 평균 fitness: 156.83
현재 세대 683 최고 해 153
population 평균 fitness: 156.92
현재 세대 684 최고 해 153
population 평균 fitness: 156.93
현재 세대 685 최고 해 153
population 평균 fitness: 157.35
현재 세대 686 최고 해 153
population 평균 fitness: 156.6
현재 세대 687 최고 해 153
po

[[7, 9, 5, 11, 3, 10, 8, 4, 6, 12, 7, 11, 6, 5, 1, 2, 3, 7, 2, 5, 4, 12, 6, 1, 8, 10, 3, 11, 6, 1, 8, 4, 2, 9, 8, 4, 10, 12, 3, 9, 7, 10, 11, 12, 2, 5, 9, 1], ['M2', 'M1', 'M3', 'M4', 'M4', 'M2', 'M3', 'M3', 'M2', 'M1', 'M1', 'M1', 'M3', 'M4', 'M2', 'M1', 'M4', 'M2', 'M3', 'M1', 'M4', 'M1', 'M1', 'M2', 'M4', 'M2', 'M3', 'M4', 'M1', 'M2', 'M3', 'M2', 'M4', 'M1', 'M3', 'M2', 'M1', 'M4', 'M2', 'M1', 'M4', 'M3', 'M2', 'M1', 'M2', 'M1', 'M4', 'M3'], 153, '3', 1585, 0.8063439065108514]
[[153, 1557, 0.8465346534653465], [151, 1537, 0.8452579034941764], [153, 1613, 0.8314049586776859], [150, 1607, 0.839527027027027], [152, 1505, 0.8480801335559266], [155, 1596, 0.8426229508196721], [152, 1575, 0.8580968280467446], [156, 1618, 0.8420195439739414], [156, 1670, 0.8292682926829268], [152, 1617, 0.8443708609271523], [155, 1629, 0.835236541598695], [153, 1566, 0.8166666666666667], [153, 1646, 0.8174342105263158], [150, 1578, 0.8593220338983051], [154, 1630, 0.8305921052631579], [153, 1654, 0.8360655